<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/KoBERT/socar_Trigger_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 설치 및 불러오기

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9MB 62kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 11.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595691 sha256=8af11b5e58c258c9b0a1ce4b55017012270221066a547fe53afea3548ac697fd
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 10.4MB/s 
     |████████████████████████████████| 757kB 11.4MB/s 
     |████████████████████████████████| 870kB 24.6MB/s 
     |████████████████████████████████| 3.0MB 40.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b77fc39f5ab32f879b5956d993d4c319c43cdd522dc2cae348c94f30335d3d4a
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f44

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kctvphl7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kctvphl7
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=717cb6df8f0a0121ed11b0c336c7f9040527a830a96589beb62a1a0ab2d367fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-8hsbz80q/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
print(vocab)

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


### 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
#dataset = pd.read_table('/content/drive/MyDrive/Trigger_list.txt',sep='\t',header=None,names=['id','문의사항', 'Trigger'])
dataset = pd.read_table('/content/drive/MyDrive/Trigger_list_31_46.txt',sep='\t',header=None,names=['id','문의사항', 'Trigger'])

dataset.head()

,id,문의사항,Trigger
0,9,브레이크 소음,0
1,9,브레이크 제보,0
2,9,TPMS 경고등,0
3,9,반납되지 않아 인입,0
4,9,브레이크 소음 발생,0


In [ ]:
# # 문자를 수치화하기 
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(dataset['문의사항'])
# dataset['문의사항'] = encoder.transform(dataset['문의사항'])
# dataset.head()

In [ ]:
# mapping - 3차 level의 Trigger case
dataset_train = pd.read_csv('/content/drive/MyDrive/2-2._lesion_info_table.csv', encoding='cp949')
inspect_type=dataset_train['lesion_detail_type']
inspect_type=set(inspect_type.values.tolist())
mapping = dict(zip(range(len(inspect_type)), inspect_type))

In [ ]:
# 가져온 내 데이터를 훈련 테스트셋과 테스트 데이터 셋으로 나누기 
from sklearn.model_selection import train_test_split   
#dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=42)
#print("train shape is:", len(dataset_train))
#print("test shape is:", len(dataset_test))
# 튜토리얼에서 사용하는 데이터 로드 방식
# dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Trigger_list.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Trigger_list.txt", field_indices=[1,2], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Trigger_list_31_46.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Trigger_list_31_46.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
print(type(dataset_train))
for i in range(len(dataset_train)):
  print(dataset_train[i])

<class 'gluonnlp.data.dataset.TSVDataset'>
['브레이크 제보', '0']
['TPMS 경고등', '0']
['반납되지 않아 인입', '0']
['브레이크 소음 발생', '0']
['브레이크 소음 발생', '0']
['브레이크 소음 문의', '0']
['브레이크 소음 발생', '0']
['주행 중 소음으로 인입', '0']
['브레이크 소음 확인 필요', '0']
['차량 사진 보내고 인입', '0']
['운행 중 소음으로 인입', '0']
['브레이크 소음 및 떨림', '0']
['상세 사유 브레이크 고장', '0']
['브레이크 소음으로 인입', '0']
['브레이크 갈리는 소리 들림', '0']
['브레이크 갈리는 소리가 남', '0']
['내부 상태 불량 제보 차 인입', '0']
['브레이크 잡을 때 소음 발생', '0']
['브레이크 밟으면 소음 발생', '0']
['브레이크 작동 시 소음 발생', '0']
['차량 브레이크 하이패스 문의', '0']
['브레이크 밟을 때마다 소음 발생인 입', '0']
['브레이크 밟으면 덜덜거린다고 함', '0']
['브레이크가 밟으면 소리가 난다며 인입', '0']
['브레이크 소리가 크게 들린다고 함', '0']
['브레이크 제동에 문제 있는 것 같다 함', '0']
['브레이크 밟을 때 소음 발생으로 인입', '0']
['상세 사유 브레이크에서 끼익 소리가 남', '0']
['브레이크 기계 끌 리 소리처럼 난다고 함', '0']
['고객 브레이크 소음으로 인한 문의', '0']
['문 제어 요청 양식 예약 메모 콜라비', '0']
['브레이크 밟으면 앞바퀴 소음 발생', '0']
['고객님 차량 주행 시 드르륵 소음 발생인 입', '0']
['브레이크 밟을 때마다 끽 소음 발생인 입', '0']
['브레이크 밟을 때 끼익 소리가 난 다시', '0']
['브레이크 밀림 쇠 갈리는 소리로 인입', '0']
['브레이크 밟을 때 소음 발생되어 문의', '0']
['브레이크 밟으면 

### 모델학습

In [ ]:
## Setting parameters
max_len = 164  # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32 # 본 64
warmup_ratio = 0.1
num_epochs = 180
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
 
    # train 데이터 학습
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test 데이터 학습
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.007667064666748 train acc 0.03125

epoch 1 train acc 0.0520979020979021


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.024912587412587412


...

epoch 41 batch id 1 loss 4.295675754547119 train acc 0.0

epoch 41 train acc 0.02954545454545454



epoch 41 test acc 0.15887237762237763


epoch 42 batch id 1 loss 4.1201090812683105 train acc 0.0

epoch 42 train acc 0.02814685314685315



epoch 42 test acc 0.13221153846153846


epoch 43 batch id 1 loss 4.146862983703613 train acc 0.0


In [ ]:
unseen_test = pd.DataFrame([['타이어 파스', 7]], columns = [['질문 내용', '질병명']])
unseen_values = unseen_test.values
 
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
 
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)